In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import re
import optuna
from optuna import Trial, visualization
from sklearn.model_selection import StratifiedKFold, KFold,cross_val_predict,cross_val_score,cross_validate
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV


# Load train and test data
train_data = pd.read_csv('/kaggle/input/primal2-0/final_train.csv')
test_data = pd.read_csv('/kaggle/input/primal2-0/final_test.csv')

# Concatenate train and test data for preprocessing
combined_data = pd.concat([train_data, test_data], ignore_index=True)

value_mapping = {
    'a': 6,  # Example: 10 lakhs
}
# Replace values in the 'ticket size' column using the mapping
combined_data['Previous_Organizations'] = test_data['Previous_Organizations'].replace(value_mapping)

# Drop specified columns
combined_data.drop(columns=['Designation', 'DOJ', 'R_District', 'B_District', 'Company_1', 'Company_2', 'Skill'], inplace=True)

# Replace Graduation and Qualification values
combined_data['Graduation'] = combined_data['Graduation'].fillna(0).map({'full time': 1, 'part time': 0, 0: 0})
combined_data['Qualification'] = combined_data['Qualification'].map({'graduate': 2, 'under graduate': 1, 'post graduate': 3, 'others': 0, 'diploma holders': 4}).fillna(-1)

# Set 'Previous_Organizations' as type int, treating NaN as 0
combined_data['Previous_Organizations'] = combined_data['Previous_Organizations'].fillna(0).astype(int)

# One-hot encoding for categorical variables
columns_to_encode = ['Industry', 'Source', 'Department', 'R_Region', 'B_Region']
combined_data_encoded = pd.get_dummies(combined_data, columns=columns_to_encode)

# Create indicator variables for states
states = list(combined_data['R_State']) + list(combined_data['B_State'])
for state in states:
    combined_data_encoded[state] = ((combined_data['R_State'] == state) | (combined_data['B_State'] == state)).astype(int)

# Drop 'R_State' and 'B_State'
combined_data_encoded.drop(columns=['R_State', 'B_State', 'Unnamed: 0'], inplace=True)
combined_data_encoded.columns = [re.sub(r'\W+', '_', col) for col in combined_data_encoded.columns]

# Separate back into train and test data
train_data_processed = combined_data_encoded.iloc[:len(train_data)]
test_data_processed = combined_data_encoded.iloc[len(train_data):].drop(columns='Performance')  # Assuming 'Target' is the target column

In [2]:
df_all = train_data_processed
df_test = test_data_processed
df_submission = pd.read_csv("/kaggle/input/sample/sample.csv")

In [3]:


X = df_all.drop(labels=['CandidateID','Performance'],axis=1,inplace=False).copy()
y = df_all['Performance'].values
X_test = df_test.drop(labels=['CandidateID'],axis=1,inplace=False).copy()
random_seed = 0



In [4]:
encoder = OneHotEncoder()
all_encoded = encoder.fit_transform(pd.concat([X,X_test]))
#X = all_encoded[0:len(X)]
#X_test = all_encoded[len(X):]
X = all_encoded.tocsr()[0:len(X)]
X_test = all_encoded [len(df_all):]

In [5]:
df_all['Performance'].value_counts()

Performance
0.0    450
1.0    295
Name: count, dtype: int64

In [6]:
X = df_all.drop(labels=['CandidateID','Performance'],axis=1,inplace=False).copy()
y = df_all['Performance'].values
X_test = df_test.drop(labels=['CandidateID'],axis=1,inplace=False).copy()
random_seed = 0

In [7]:
encoder = OneHotEncoder()
all_encoded = encoder.fit_transform(pd.concat([X,X_test]))
#X = all_encoded[0:len(X)]
#X_test = all_encoded[len(X):]
X = all_encoded.tocsr()[0:len(X)]
X_test = all_encoded [len(df_all):]

In [8]:
from sklearn.metrics import f1_score

def objective(trial, data=X, target=y):
    param_space = {
        'C': trial.suggest_float('C', 0.01, 100),
        'loss': trial.suggest_categorical('loss', ['hinge','squared_hinge']),
        'penalty': trial.suggest_categorical('penalty', ['l2']),
        'dual': trial.suggest_categorical('dual', [True]),
        'multi_class': trial.suggest_categorical('multi_class', ['crammer_singer', 'ovr'])
    }
    k=5
    seed_list=[0]
    kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=random_seed)
    oof = np.zeros((len(df_all),2))
    score_list = []
    fold=1
  
    splits = list(kf.split(X,y))
    for train_idx, val_idx in splits:
        X_train, X_val = X[train_idx,:], X[val_idx,:]
        y_train, y_val = y[train_idx], y[val_idx]

        val_preds_list = []

        for seed in seed_list:
          # fit and run model
          param_space['random_state'] = seed

          base_model = LinearSVC(**param_space, max_iter=5000)
          model = CalibratedClassifierCV(base_model, method='sigmoid', cv=k)
          model.fit(X_train,y=y_train)

    
          val_preds_list.append(model.predict_proba(X_val))
          #print(model.predict_proba(X_val))
     #test_preds_list.append(model.predict_proba(X_test)[:,1])
    
        oof[val_idx] = np.mean(val_preds_list,axis=0)
        score = f1_score(y_val, np.argmax(oof[val_idx], axis=1), average='weighted')
        print(f"fold: {fold},F1 Score: {score}")
        score_list.append(score)
        fold += 1
    cv_score = np.mean(score_list)
    
    return cv_score

In [9]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials= 200)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-04-11 05:43:15,015] A new study created in memory with name: no-name-d3b884d3-fe07-4f8a-9b45-de12c13c2292


fold: 1,F1 Score: 0.4952299354698326
fold: 2,F1 Score: 0.4876272099016
fold: 3,F1 Score: 0.4775187051354408
fold: 4,F1 Score: 0.541544520257005


[I 2024-04-11 05:43:51,714] Trial 0 finished with value: 0.4989920145568334 and parameters: {'C': 15.586453105331058, 'loss': 'squared_hinge', 'penalty': 'l2', 'dual': True, 'multi_class': 'crammer_singer'}. Best is trial 0 with value: 0.4989920145568334.


fold: 5,F1 Score: 0.4930397020202887
fold: 1,F1 Score: 0.4952299354698326
fold: 2,F1 Score: 0.4876272099016
fold: 3,F1 Score: 0.4775187051354408
fold: 4,F1 Score: 0.541544520257005


[I 2024-04-11 05:44:06,146] Trial 1 finished with value: 0.4989920145568334 and parameters: {'C': 40.54388173250089, 'loss': 'squared_hinge', 'penalty': 'l2', 'dual': True, 'multi_class': 'ovr'}. Best is trial 0 with value: 0.4989920145568334.


...

fold: 5,F1 Score: 0.4930397020202887
fold: 1,F1 Score: 0.4952299354698326
fold: 2,F1 Score: 0.4876272099016
fold: 3,F1 Score: 0.4775187051354408
fold: 4,F1 Score: 0.541544520257005


[I 2024-04-11 06:30:18,779] Trial 198 finished with value: 0.4989920145568334 and parameters: {'C': 4.909617523238015, 'loss': 'squared_hinge', 'penalty': 'l2', 'dual': True, 'multi_class': 'ovr'}. Best is trial 101 with value: 0.568440583700561.


fold: 5,F1 Score: 0.4930397020202887
fold: 1,F1 Score: 0.4952299354698326
fold: 2,F1 Score: 0.4876272099016
fold: 3,F1 Score: 0.4775187051354408
fold: 4,F1 Score: 0.541544520257005


[I 2024-04-11 06:30:30,774] Trial 199 finished with value: 0.4989920145568334 and parameters: {'C': 2.0783826712539253, 'loss': 'squared_hinge', 'penalty': 'l2', 'dual': True, 'multi_class': 'ovr'}. Best is trial 101 with value: 0.568440583700561.


fold: 5,F1 Score: 0.4930397020202887
Number of finished trials: 200
Best trial: {'C': 0.010361948404221305, 'loss': 'squared_hinge', 'penalty': 'l2', 'dual': True, 'multi_class': 'ovr'}


In [10]:
params = study.best_params
base_model = LinearSVC(**params,max_iter=5000)
model = CalibratedClassifierCV(base_model, method='sigmoid')
model.fit(X,y)
pred = model.predict_proba(X_test)

In [11]:
df_test[['class1','class2']] = pred

submission = df_test[['CandidateID','class1','class2']]
submission.to_csv('linearsvc.csv')